In [1]:
!pip install kaggle

In [2]:
#Upload your own kaggle account's json file
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"piyusrout","key":"71f444a7f1387e811778bef18826b18c"}'}

In [3]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d tongpython/cat-and-dog

 96% 209M/218M [00:04<00:00, 42.2MB/s]
100% 218M/218M [00:04<00:00, 51.5MB/s]


In [ ]:
!unzip cat-and-dog.zip

In [6]:
!rm cat-and-dog.zip

In [ ]:
!pip install livelossplot

In [10]:
import os
import shutil
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D , MaxPool2D , Flatten , Dense , Activation , Dropout
from keras.optimizers import Adam
from keras.models import Sequential
from keras.callbacks import TensorBoard
from livelossplot import PlotLossesKeras

In [11]:
if not os.path.isdir("train/"):
  os.mkdir("train")
  shutil.move("training_set/training_set/cats" , "train/")
  shutil.move("training_set/training_set/dogs" , "train/")

if not os.path.isdir("test/"):
  os.mkdir("test")
  shutil.move("test_set/test_set/cats" , "test/")
  shutil.move("test_set/test_set/dogs" , "test/")

In [12]:
!rm -r training_set
!rm -r test_set

In [13]:
train_dir = "train"
train_cats = os.listdir("train/cats")
train_dogs = os.listdir("train/dogs")

print("There are " + str(len(train_cats)) + " cat samples.")
print("There are " + str(len(train_dogs)) + " dog samples.")

There are 4001 cat samples.
There are 4006 dog samples.


In [14]:
test_dir = "train"
test_cats = os.listdir("test/cats")
test_dogs = os.listdir("test/dogs")

print("There are " + str(len(test_cats)) + " cat samples.")
print("There are " + str(len(test_dogs)) + " dog samples.")

There are 1012 cat samples.
There are 1013 dog samples.


In [15]:
img_size = 128
batch_size = 64

datagen_train = ImageDataGenerator(shear_range=0.2,
                                   rescale=1./255.,
                                   horizontal_flip=False)
train_generator = datagen_train.flow_from_directory('train',
                                                    target_size=(img_size , img_size),
                                                    batch_size=batch_size,
                                                    color_mode='rgb',
                                                    class_mode='categorical',
                                                    shuffle=True)

Found 8005 images belonging to 2 classes.


In [16]:
img_size = 128
batch_size = 64

datagen_test = ImageDataGenerator(shear_range=0.2,
                                   rescale=1./255.,
                                   horizontal_flip=False)
test_generator = datagen_test.flow_from_directory('test',
                                                    target_size=(img_size , img_size),
                                                    batch_size=batch_size,
                                                    color_mode='rgb',
                                                    class_mode='categorical',
                                                    shuffle=False)

Found 2023 images belonging to 2 classes.


In [17]:
model = Sequential()

#Layer 1
model.add(Conv2D(64 , (3 , 3) , padding='same' , activation='relu' , input_shape = (img_size , img_size , 3)))
model.add(MaxPool2D(pool_size=(2 , 2)))
#model.add(Dropout(0.25))

#Layer 2
model.add(Conv2D(64 , (3 , 3) , padding='same' , activation='relu'))
model.add(MaxPool2D(pool_size=(2 , 2)))
#model.add(Dropout(0.25))

#Layer 3
#model.add(Conv2D(128 , (3 , 3) , padding='same' , activation='relu'))
#model.add(MaxPool2D(pool_size=(2 , 2)))
#model.add(Dropout(0.25))

model.add(Flatten())

#FC1
model.add(Dense(units=128 , activation='relu'))
#model.add(Dropout(0.5))

#FC2
model.add(Dense(units=2 , activation='softmax'))

optimizer = Adam(learning_rate=0.005)
model.compile(optimizer=optimizer , loss='categorical_crossentropy' , metrics=['accuracy'])

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 128, 128, 64)      1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 64, 64, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 65536)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               8388736   
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 2

In [ ]:
epochs=10
steps_per_epoch=train_generator.n // train_generator.batch_size
validation_steps=test_generator.n // test_generator.batch_size

history = model.fit(x=train_generator,
          steps_per_epoch=steps_per_epoch,
          epochs=epochs,
          validation_data=test_generator,
          validation_steps=validation_steps)